### Imports

In [1]:
import os
import streamlit as st
from pprint import pprint
import sys
import io
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from langchain_community.callbacks import get_openai_callback
import matplotlib.pyplot as plt
import json
import os
import re
import logging
from pprint import pprint
from typing import List

def load_environment_variables():
    """Load environment variables from a .env file."""
    load_dotenv()
    os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
    os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./credentials.json"
    os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
    os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


load_environment_variables()

from agents.Chart_Agent import chart_generator
from agents.SQLAgent import SQLAgent
from agents.TableMaker import TableMaker
from agents.FDD_Agents import executive_agent_mode,key_metrics_agent_mode,business_model_agent_mode
from agents.FDD_generator import FDD_Generator_handeler
from agents.visual_json import get_json
from agents.Disc_reframer import Disc_question_reframer
from agents.answer_aggregator import aggregator
from agents.answer_grader import answer_grader
from agents.document_relevent_router import relevency_router
from agents.finance_react_agent import finance_react_agent
from agents.finance_agent import finance_agent
from agents.hallucination_grader import hallucination_grader
from agents.main_router import question_router
from agents.query_decomposition import decomposer
from agents.question_rewritter import question_rewriter
from agents.reasoning_agent import reasoner
from agents.rectifier import rectifier
from agents.retrieval_grader import retrieval_grader
from agents.verification_agent import verifier
from langchain.schema import Document
from langgraph.graph import END, START, StateGraph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
from rag.rag import (
    compress_documents,
    create_compressor,
    create_groq_llm,
    create_openai_llm,
    create_pathway_client,
    create_prompt_template,
    get_answer,
    retrieve_relevant_documents,
)
from tools.bar_maker import generate_bar_chart
from tools.line_maker import generate_line_chart
from tools.pie_maker import generate_pie_chart
from tools.tools import web_search_tool
from typing_extensions import TypedDict

llm = create_openai_llm()
prompt = create_prompt_template()
BASELINE_VERIFICATION_QUESTIONS = []

# need to add this as a state value in workflow, havent done it yet
NOT_SUPPORTED_COUNTER = 0
WEB_SEARCH_COUNTER = 0

/home/lalit/miniconda3/envs/pathway/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


get_latest_key_metrics
I am here at get_latest_key_metrics
h1234521: ratios [{'name': 'P/E Ratio', 'value': 23.907916525509858}, {'name': 'P/B Ratio', 'value': 6.225883710507836}, {'name': 'Debt to Equity Ratio', 'value': 0.09073714001390364}, {'name': 'Free Cash Flow', 'value': 5.502375296912114}, {'name': 'PEG Ratio', 'value': 0.877898694816722}, {'name': 'Working Capital Ratio', 'value': 2.096584936563424}, {'name': 'Quick Ratio', 'value': 2.096584936563424}, {'name': 'Earnings Ratio', 'value': 0.24006649446638517}, {'name': 'Return on Equity', 'value': 0.26041096905557576}]
[{'name': 'P/E Ratio', 'value': 23.907916525509858}, {'name': 'P/B Ratio', 'value': 6.225883710507836}, {'name': 'Debt to Equity Ratio', 'value': 0.09073714001390364}, {'name': 'Free Cash Flow', 'value': 5.502375296912114}, {'name': 'PEG Ratio', 'value': 0.877898694816722}, {'name': 'Working Capital Ratio', 'value': 2.096584936563424}, {'name': 'Quick Ratio', 'value': 2.096584936563424}, {'name': 'Earnings Ratio

In [2]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
        max_revision: int
    """

    question: str
    generation: str
    documents: List[str]
    revision_number: int
    max_revisions: int
    final_generation: str
    first_attempt: bool

tokens=0
inp_tok=0

async def retrieve(state):
    """
    Retrieve documents based on the current state.

    Args:
        state (dict): The current graph state.

    Returns:
        dict: Updated state with the key "documents" containing retrieved and compressed documents.
    """
    if "revision_number" not in state or "question" not in state:
        raise ValueError("State must contain 'revision_number' and 'question' keys.")

    question = state["question"]

    # Initialize token tracking
    global tokens
    global inp_tok
    tok = 0
    pr_tok = 0
    try:

        # if state.get("revision_number", 0) == 0:
        print("---RETRIEVE---")
        #print("Question",state["question"])
        # Retrieval
        retriever = await create_pathway_client()
        compressor = create_compressor()

        tok, pr_tok, compressed_docs = compress_documents(retriever, question, compressor)
        print(compressed_docs)
        tokens += tok
        inp_tok += pr_tok
        print("Len Docs:",len(compressed_docs))
        # if len(compressed_docs)==0:
        #     state["revision_number"] + 1
        #     return web_search(state)
        if not compressed_docs:
            print("No documents retrieved or compressed.")

        return {
            "documents": compressed_docs,
            "question": question,
            # "revision_number": state["revision_number"] + 1,
            "revision_number": state["revision_number"],
            "max_revisions": 1,
            "first_attempt": state["first_attempt"]
        }
        # elif state["revision_number"] > 0:
        #     retriever = await create_pathway_client()
        #     relevant_docs = retrieve_relevant_documents(retriever, question)
        #     # print("RETRIEVE-RELEVANT: ", len(relevant_docs), "---", relevant_docs)
        #     if not relevant_docs:
        #         print("No documents retrieved.")
        #     state["documents"].extend(relevant_docs)
        #     return {
        #         "documents": state['documents'],
        #         "question": question,
        #         # "revision_number": state["revision_number"] + 1,
        #         "revision_number": state["revision_number"],
        #         "max_revisions": 1,
        #     }

    except:
        #print("RETRIEVE-WEB")
        return web_search(state)
    
    # else:
    #     print("---Dynamic-RETRIEVE---")

    #     if "documents" not in state:
    #         raise ValueError("State must contain 'documents' for dynamic retrieval.")

    #     # Dynamic retrieval based on pre-retrieved documents
    #     retriever = create_pathway_client(k=1)
    #     compressor = create_compressor()
    #     pre_retrieved_docs = [doc.page_content for doc in state["documents"]]
    #     updated_docs = state["documents"].copy()

    #     for doc in pre_retrieved_docs:
    #         context_guided_query = f"{question}\ncontext: {doc}"
    #         tok, pr_tok, additional_docs = compress_documents(retriever, context_guided_query, compressor)
    #         tokens += tok
    #         inp_tok += pr_tok
    #         updated_docs.extend(additional_docs)

    #     if not updated_docs:
    #         print("No dynamic documents retrieved or compressed.")

    #     return {
    #         "documents": updated_docs,
    #         "question": question,
    #         "revision_number": state["revision_number"],
    #         "max_revisions": 1,
    #     }


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    print("Question",state["question"])
    global tokens, inp_tok

    question = state["question"]
    documents = state["documents"]
    print(f"In GETANSWER {documents}")

    # Get answer and update token counts
    # final_answer='\n'.join([doc.page_content for doc in documents])
    # final_answer_pairs = "###Document::\n".join([f'content: {doc.page_content} || doc name :{doc.metadata["name"]} || page number: {doc.metadata["page_number"]}' for doc in documents])
    final_answer_pairs = ""
    for doc in documents:
        try:
            final_answer_pairs += "###Document::\n" + f'content: {doc.page_content} || doc name :{doc.metadata["name"]} || page number: {doc.metadata["page_number"]}'
        except:
            final_answer_pairs += "###Document::\n" + f'content: {doc.page_content} || metadata: {doc.metadata}'
        
        
    tok, pr_tok, generation = get_answer(final_answer_pairs, question, llm, prompt)
    tokens += tok
    inp_tok += pr_tok
    print("Generate node output: ", generation)
    print(f"Total tokens: {tok}")
    print(f"Prompt tokens: {pr_tok}")
    print("First Generation: ", state["first_attempt"])

    return {
        "documents": documents,
        "question": question,
        "final_generation": generation,
        "revision_number": state["revision_number"],
        "max_revisions": 1,
        "first_attempt": state["first_attempt"]
    }



def grade_documents(state):
    global tokens
    global inp_tok
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    print("Question",state["question"])
    question = state["question"]
    documents = state["documents"]

    # Score each document
    filtered_docs = []
    for doc in documents:
        with get_openai_callback() as cb:
            score = retrieval_grader.invoke({"question": question, "document": doc.page_content})
        tokens += cb.total_tokens
        inp_tok += cb.prompt_tokens
        print(cb.total_tokens)
        print(cb.prompt_tokens)
        
        if score.binary_score == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(doc)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")

    if state["revision_number"] == state["max_revisions"]:
        print("----MAXIMUM REVISIONS REACHED, NOT FILTERING DOCUMENTS----")
        return {"documents": documents, "question": question}

    return {
        "documents": filtered_docs,
        "question": question,
        "revision_number": state["revision_number"],
        "max_revisions": 1,
        "first_attempt": state["first_attempt"]
    }


def transform_query(state):
    global tokens
    global inp_tok
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    print("Question",state["question"])
    question = state["question"]
    documents = state["documents"]

    # Re-write the question
    with get_openai_callback() as cb:
        better_question = question_rewriter.invoke({"question": question})
        state["revision_number"] += 1

    # Update token counts
    tokens += cb.total_tokens
    inp_tok += cb.prompt_tokens

    print(f"Total tokens: {cb.total_tokens}")
    print(f"Prompt tokens: {cb.prompt_tokens}")
    
    if state["first_attempt"] == True:
        state["first_attempt"] = False

    return {
        "documents": documents,
        "question": better_question,
        "revision_number": state["revision_number"],
        "max_revisions": 1,
        "first_attempt": state["first_attempt"]
    }


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    question = state["question"]
    state["documents"]=state.get("documents",[])
    
    for doc in state["documents"]:
        if doc.metadata.get("source")=="web_search":
           return state 
    
    print("---WEB SEARCH---")
    print("Question",state["question"])
        
    # Perform web search
    search_results = web_search_tool.invoke({"query": question})
    
    # Combine search results into a single document
    try:
        combined_results = "\n".join([result["content"] for result in search_results])
    except Exception as e:
        print(f"Error combining search results: {e}")
        print("Search results: ", search_results)
        combined_results = ""
    web_document = Document(page_content=combined_results, metadata={"source": "web_search"})
    state["documents"].extend([web_document])
    # print("DOCUMENTS-WEB_SEARCH",state["documents"])
    global WEB_SEARCH_COUNTER
    WEB_SEARCH_COUNTER += 1
    return {
        "documents": state["documents"], 
        "question": question,
        "revision_number": state["revision_number"],
        "max_revisions": 1,
        "first_attempt": state["first_attempt"]
    }


### Edges ###


def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    print("Question",state["question"])
    global tokens, inp_tok
    question = state["question"]

    # Determine routing using question_router with token tracking
    with get_openai_callback() as callback:
        routing_result = question_router.invoke({"question": question})
        
        # Update token counters
        tokens += callback.total_tokens
        inp_tok += callback.prompt_tokens
        
        # Log token usage
        print(f"Total tokens: {callback.total_tokens}")
        print(f"Prompt tokens: {callback.prompt_tokens}")

    # Route based on determined data source
    if routing_result.datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    else:  # vectorstore case
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"

def decide_to_retrieve(state):
    """
    Determines whether to retrieve a question (compressed or without compression handled inside retriever) 
    or web-search
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """
    print("---Decide To Review---")
    print("Question",state["question"])
    if state["revision_number"] <= 1:
        return "retrieve"
    else:
        return "web_search"
    
    
def decide_to_generate(state):
    """
    Determines whether to generate an answer, re-generate a question, 
    or route to financial or SQL agents based on the document structure.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    global tokens
    global inp_tok
    print("---ASSESS GRADED DOCUMENTS---")
    print("Question",state["question"])
    question = state["question"]
    filtered_documents = state["documents"]
    # print("filtered documents: ", filtered_documents)
    filtered_documents = [doc.page_content for doc in filtered_documents]
    
    with get_openai_callback() as cb:
        answerability = relevency_router.invoke({"question": question, "context": filtered_documents})
    
    tokens += cb.total_tokens
    inp_tok += cb.prompt_tokens
    
    print(cb.total_tokens)
    print(cb.prompt_tokens)
    print("Answerability: ", answerability)
    # print("answerability.datasource == answerable:", answerability.datasource == "answerable")
    # print("answerability.datasource == not_answerable:", answerability.datasource == "not_answerable")
    
    if filtered_documents and answerability.datasource != "answerable" and state["first_attempt"]==False:
        print("---DECISION: REDIRECTING TO FINANCE AGENT ---")
        return "not_answerable"

    # Check if all documents are filtered out and if more revisions are allowed
    if not filtered_documents and state["revision_number"] < state["max_revisions"]:
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---")
        return "transform_query"

    # Check for multiple tables in "text_as_html"
    total_table_count = 0
    for doc in filtered_documents:
        if "text_as_html" in doc:
            soup = BeautifulSoup(doc["text_as_html"], "html.parser")
            total_table_count += len(soup.find_all("table"))

    if total_table_count >= 2:
        print("---DECISION: MULTIPLE TABLES DETECTED, ROUTING TO SQL_AGENT---")
        return "sql_agents"

    # We have relevant documents, so generate answer
    print("---DECISION: GENERATE---")
    return "generate"


def grade_generation_v_documents_and_question(state):
    global tokens
    global inp_tok
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    print("Question",state["question"])
    global tokens, inp_tok
    question = state["question"]
    documents = state["documents"]
    documents_content = [doc.page_content for doc in documents]
    generation = state["final_generation"]

    # Check for hallucinations
    with get_openai_callback() as cb:
        score = hallucination_grader.invoke({"documents": documents_content, "generation": generation})
        hallucination_grade = score.binary_score
    tokens += cb.total_tokens
    inp_tok += cb.prompt_tokens
    print(cb.total_tokens)
    print(cb.prompt_tokens)

    if hallucination_grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        
        # Check if the generation answers the question
        print("---GRADE GENERATION vs QUESTION---")
        with get_openai_callback() as cb:
            score = answer_grader.invoke({"question": question, "generation": generation})
            answer_grade = score.binary_score
        tokens += cb.total_tokens
        inp_tok += cb.prompt_tokens
        print(cb.total_tokens)
        print(cb.prompt_tokens)

        if answer_grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            if state["first_attempt"] == True:
                return "not useful"
                
            if state["revision_number"] >= state["max_revisions"]:
                print("---DECISION: MAX REVISIONS REACHED, STOPPING---")
                return "stop"
            else:
                print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
                return "not useful"
    else:
        if state["first_attempt"] == True:
            return "not useful"
        
        if state["revision_number"] >= state["max_revisions"]:
            print("---DECISION: MAX REVISIONS REACHED, STOPPING---")
            return "stop"
        else:
            global NOT_SUPPORTED_COUNTER
            NOT_SUPPORTED_COUNTER += 1
            if NOT_SUPPORTED_COUNTER >= 3:
                print("---DECISION: TOO MANY UNSUPPORTED GENERATIONS, STOPPING---")
                return "stop" #We changed it because as sosn as main roter routes a query to web_search and the retrieved content doesn't satisfy for 3 continious web search rather than going to financial agent it stops and generate irrelevant result. 
                # return "financial_agent"
            print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRYING---")
            return "not supported"


def finance_tool_agent(state):
    """
    Call finance agent tools.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---FINANCE AGENT---",state["question"])
    question = state["question"]
    documents = state["documents"]
    global tokens
    global inp_tok

    # Invoke the finance react agent
    # with get_openai_callback() as cb:
    #     generation = finance_react_agent.invoke({"input": question})
    
    # # Update token counts
    # tokens += cb.total_tokens
    # inp_tok += cb.prompt_tokens
    # print(cb.total_tokens)
    # print(cb.prompt_tokens)
    
    # print("Finance react agent generation: ", generation)
    
    # # Extract the final generation output
    # final_generation = generation["output"]
    
    # Invoke the finance agent
    with get_openai_callback() as cb:
        generation = finance_agent.invoke(question)
        web_search_flag = False
        for gen in generation:
            # print("GEN",gen)
            try:
                if "Will be right back" in gen['output'][0][0]:
                  web_search_flag = True
            except Exception as e:
                pass
            try:
                if "Failed to get" in gen['output']:
                  web_search_flag = True
            except Exception as e:
                pass
        # print("GENERATION",generation)
    tokens += cb.total_tokens
    inp_tok += cb.prompt_tokens
    print(cb.total_tokens)
    print(cb.prompt_tokens)
    
    global WEB_SEARCH_COUNTER
    if web_search_flag and WEB_SEARCH_COUNTER == 0:
        generation.append({"output": web_search(state)["documents"][-1].page_content})
    
    final_generation = [item['output'] for item in generation]

    if len(final_generation)==0:
        final_generation = [web_search(state)["documents"][-1].page_content]
        
    final_generation_string=f"{final_generation}"
    # print("QUESTION",state["question"],"DOCUMENTS",final_generation_string)
    # print("Documents: ", documents)
    # documents.append(Document(page_content=final_generation_string, metadata={"source":"finance_agent_output"}))
    return {
        "documents": documents,
        "question": question,
        "generation": final_generation_string,
        "revision_number": state["revision_number"], 
        "max_revisions": 1
    }


def chart_Agents(generation):
    print("---CHART CREATING AGENT---")

    # Generate the chart reasoning response
    chart_data_response = chart_generator.invoke({"generation": generation})
    print(f"Chart Data Response: {chart_data_response}")

    # Check if the response indicates no chart is possible
    if "No chart possible" in chart_data_response:
        print("No chart can be generated from this context.")
        return "No chart created."

    try:
        # Extract the JSON from the response using regex
        json_match = re.search(r"\{.*\}", chart_data_response, re.DOTALL)
        if not json_match:
            raise ValueError("No valid JSON found in the response.")

        json_text = json_match.group(0)  # Extract the JSON string
        chart_data = json.loads(json_text)  # Parse the JSON string

        # Determine chart type and delegate to the appropriate function
        chart_type = chart_data.get("chartType", "").lower()

        # Generate a unique file path
        save_dir = "charts"
        os.makedirs(save_dir, exist_ok=True)

        # Check if file already exists and create a unique name
        base_filename = "chart.png"
        save_path = os.path.join(save_dir, base_filename)
        counter = 1
        while os.path.exists(save_path):
            save_path = os.path.join(save_dir, f"chart_{counter}.png")
            counter += 1

        if chart_type == "bar":
            return generate_bar_chart(chart_data, save_path)
        elif chart_type == "line":
            return generate_line_chart(chart_data, save_path)
        elif chart_type == "pie":
            return generate_pie_chart(chart_data, save_path)
        else:
            raise ValueError(f"Unsupported chart type: {chart_type}")

    except json.JSONDecodeError as jde:
        print(f"JSON parsing error: {jde}")
        return "Failed to create chart due to JSON parsing error."
    except Exception as e:
        print(f"Error in chart generation: {e}")
        return "Failed to create chart."


def reasoning_agent(state):
    """
    Reason based on the question and documents.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---REASON AGENT---")
    print("Question",state["question"])
    question = state["question"]
    documents = state["documents"]
    finance_agent_generation = state["generation"]
    # docs_content = [doc.page_content for doc in documents]
    global tokens, inp_tok

    with get_openai_callback() as cb:
        reasoning_output = reasoner.invoke({"question": question, "documents": finance_agent_generation})
        # print(f"Reasoning Output: {reasoning_output}")
        # print("Documents: ", documents)
        new_document = Document(page_content=reasoning_output, metadata={"source":"web search or finance tools"})
        state['documents'].append(new_document)

        if any(keyword in reasoning_output.lower() for keyword in ["chart", "graph", "visualize", "plot"]):
            print("Reasoning indicates a chart may need to be created. Trying to create the chart\n")
            save_path = chart_Agents(reasoning_output)
            if save_path is not None:
                print(f"Saved the image in: {save_path}")

    tokens += cb.total_tokens
    inp_tok += cb.prompt_tokens
    print(cb.total_tokens)
    print(cb.prompt_tokens)

    return {
        "documents": state['documents'],
        "question": question,
        "revision_number": state["revision_number"],
        "max_revisions": 1
    }


def sql_agents(state):
    """
    Handle SQL-based questions by processing metadata and querying the database.

    Args:
        state (dict): The current graph state.

    Returns:
        state (dict): Updated state with the SQL query result.
    """
    print("---SQL QUERY---")
    print("Question",state["question"])
    # Process metadata and populate the database
    chunk = state["document"]
    table_maker = TableMaker(db_name="testDB.db")
    table_maker.process_chunk(chunk)

    # Initialize the SQLAgent
    sql_agent = SQLAgent(
        api_key=os.getenv["GROQ_API_KEY"],
        db_path="example.db",
        google_api_key=os.getenv("GOOGLE_API_KEY")
    )

    # Ask the SQLAgent for the answer
    question = state["question"]
    answer = sql_agent.ask_question(question)
    answer_string=f"{answer}"
    # print(f"SQL Query Result: {answer}")
    state["documents"].append(Document(page_content=answer_string), metadata={"source":"sql_agent_output"})
    state["first_attempt"] = False
    return state


workflow = StateGraph(GraphState)

# Define the nodes (same as provided)
workflow.add_node("web_search", web_search)  
workflow.add_node("retrieve", retrieve) 
workflow.add_node("grade_documents", grade_documents)  
workflow.add_node("generate", generate) 
workflow.add_node("transform_query", transform_query)
workflow.add_node("finance_agent", finance_tool_agent) 
workflow.add_node("reasoning_agent", reasoning_agent) 
workflow.add_node("sql_agents", sql_agents)

# Build graph (same as provided)
# workflow.add_conditional_edges(
#     START,
#     route_question,
#     {
#         "web_search": "web_search",
#         "vectorstore": "retrieve",
#     },
# )
# Directly add an edge from START to the "retrieve" node
workflow.add_edge(START, "retrieve")

# workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents") #Based on web search it oftent directly generates output without financial tools
workflow.add_edge("web_search", "finance_agent")

workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "not_answerable": "finance_agent",
        "generate": "generate",
        "sql_agents" : "sql_agents",
    },
)

workflow.add_edge("finance_agent", "reasoning_agent")
workflow.add_edge("reasoning_agent", "generate")
workflow.add_conditional_edges(
    "transform_query",
    decide_to_retrieve,
    {
        "retrieve": "retrieve",
        "web_search": "web_search",
    },
)

workflow.add_conditional_edges(
    "sql_agents",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
        "stop": END,
    },
)

workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
        "stop": END,
    },
)

# Compile
app = workflow.compile()

### run workflow AAPL 10Q

In [3]:
import pandas as pd
question_df = pd.read_csv("./AAPL_10Q_dataset/aapl_qna_data.csv")
question_df.head()

,Question,Source Docs,Question Type,Source Chunk Type,Answer
0,How has Apple's total net sales changed over t...,*AAPL*,Multi-Doc RAG,Table,"Based on the provided documents, Apple's total..."
1,What are the major factors contributing to the...,*AAPL*,Multi-Doc RAG,Text,In the most recent 10-Q for the quarter ended ...
2,Has there been any significant change in Apple...,*AAPL*,Multi-Doc RAG,Table,"Yes, there has been a change in Apple's operat..."
3,How has Apple's revenue from iPhone sales fluc...,*AAPL*,Multi-Doc RAG,Table,The revenue from iPhone sales for Apple has fl...
4,Can any trends be identified in Apple's Servic...,*AAPL*,Multi-Doc RAG,Table,"Based on the provided documents, there is a tr..."


In [4]:
import asyncio
import json
import os
import pandas as pd

async def Generator(subQueries, verification=" "):
    
    generations = []  # To store all generations

    async def process_subquery(sub_query):
        st.write(f"#### Processing {verification} query: {sub_query}")
        st.write("-" * 50)

        count = 0
        # Run the streaming process for each query
        async for output in app.astream({"question": sub_query, "revision_number": 0, "max_revisions": 1, "first_attempt":True}):
            # Capture output generation steps
            for key, value in output.items():
                print(f"#### At Node {key}")

            # logs = logger_stream.get_logs()
            # if logs:
            #     st.sidebar.text_area("Logs", logs, height=200, max_chars=None, key=f"{sub_query}_{count}")

            # if "generation" in value:
            #     return {"query": sub_query, "generation": value["generation"]}
            count += 1
        return {"query": sub_query, "generation": value["final_generation"]}

    tasks = [process_subquery(sub_query) for sub_query in subQueries]
    generations = await asyncio.gather(*tasks)

    return generations

async def process_dataframe_queries(df, decomposer, generator, aggregator, output_dir='enhancedParse_ourAgenticRAG'):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    results = []
    
    for index, row in df.iterrows():
        if index < 11:
            continue
        
        query_input = row['Question']  # Assuming the column is named 'question'
        print(f"Processing query {index}: {query_input}")
        print("=========================================")
        
        global tokens, inp_tok
        tokens = 0
        inp_tok=0
        
        # Decompose the input query into sub-queries
        inputs = {"question": query_input, "revision_number": 0, "max_revisions": 1}
        
        with get_openai_callback() as cb:
            subQueries = decomposer.invoke(inputs).sub_queries
        
        tokens += cb.total_tokens
        inp_tok += cb.prompt_tokens
        
        # If no sub-queries, use original query
        if not subQueries:
            subQueries = [query_input]
        
        # Generate outputs for the sub-queries
        generations = await generator(subQueries, verification=" ")
        
        # Aggregate answers
        answers = [gen["generation"] for gen in generations]
        
        with get_openai_callback() as cb:
            tokens += cb.total_tokens
            inp_tok += cb.prompt_tokens
            final_answer = aggregator.invoke({
                "question": query_input, 
                "answers": ", ".join(answers)
            }).answer
        
        # Store results
        result_entry = {
            "original_query": query_input,
            "sub_queries": subQueries,
            "sub_query_results": [gen["generation"] for gen in generations],
            "final_answer": final_answer,
            "total_tokens": tokens,
            "input_tokens": inp_tok,
            "output_tokens": tokens - inp_tok
        }
        
        # Generate a safe filename
        safe_filename = f"{index}_" + "".join(x for x in query_input if x.isalnum() or x in [' ', '_']).rstrip()
        safe_filename = safe_filename[:20]  # Limit filename length
        json_path = os.path.join(output_dir, f"{index}_{safe_filename}.json")
        
        # Save individual JSON file
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(result_entry, f, indent=2, ensure_ascii=False)
        
        results.append(result_entry)
    
    return results


# Process queries (you'll need to pass your actual decomposer, generator, and aggregator)
results = await process_dataframe_queries(question_df, decomposer, Generator, aggregator)

Processing query 11: Does Apple report any significant new business acquisitions or divestitures in these 10-Qs?


2024-12-05 18:34:19.137 
  command:

    streamlit run /home/lalit/miniconda3/envs/pathway/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


---RETRIEVE---
[Document(metadata={'element_id': '78823dc66d1e3a169ce36b01de897522', 'filename': 'files', 'filetype': 'application/pdf', 'id': '1uAonJcYIgjSLPGmvA0frYfJliM5OLwlM', 'languages': ['eng'], 'lastModifyingUser': {'displayName': 'interiit.pathway.24', 'emailAddress': 'interiit.pathway.24@gmail.com', 'kind': 'drive#user', 'me': False, 'permissionId': '14505053473678289633', 'photoLink': 'https://lh3.googleusercontent.com/a-/ALV-UjW4hRisylyWntAeUYq0ONJbWcwFRK3NNE7qIyJSU0EQmEdaKA=s64'}, 'mimeType': 'application/pdf', 'modifiedTime': '2024-11-10T17:49:41.000Z', 'name': '2023 Q2 AAPL.pdf', 'page_number': 23, 'parents': ['1eSyzlOwjM2PepzCuhL-UHfbRCfybopAX'], 'path': '2023 Q2 AAPL.pdf', 'seen_at': 1733419400, 'size': '263607', 'status': 'downloaded', 'thumbnailLink': 'https://lh3.googleusercontent.com/drive-storage/AJQWtBMubcBp2Ppf1CgHzOKNGk2RQXFZRQ2GJIH_E3p-WPKMlqBL8ghY5uTaufekFqYkiri_aX2kr0aBXJZHhfNAjqxL0VmfT-syqNYRxDJgwj-I9_dE=s220', 'trashed': False, 'type': 'Title', 'url': 'htt

### run workflow AAPL 10k 2022

In [3]:
import pandas as pd
question_df = pd.read_csv("rag_benchmark_apple_10k_2022_with_context.csv")
question_df.head()

In [4]:
import asyncio
import json
import os
import pandas as pd

async def Generator(subQueries, verification=" "):
    
    generations = []  # To store all generations

    async def process_subquery(sub_query):
        st.write(f"#### Processing {verification} query: {sub_query}")
        st.write("-" * 50)

        count = 0
        # Run the streaming process for each query
        async for output in app.astream({"question": sub_query, "revision_number": 0, "max_revisions": 1, "first_attempt":True}):
            # Capture output generation steps
            for key, value in output.items():
                print(f"#### At Node {key}")

            # logs = logger_stream.get_logs()
            # if logs:
            #     st.sidebar.text_area("Logs", logs, height=200, max_chars=None, key=f"{sub_query}_{count}")

            # if "generation" in value:
            #     return {"query": sub_query, "generation": value["generation"]}
            count += 1
        return {"query": sub_query, "generation": value["final_generation"]}

    tasks = [process_subquery(sub_query) for sub_query in subQueries]
    generations = await asyncio.gather(*tasks)

    return generations

async def process_dataframe_queries(df, decomposer, generator, aggregator, output_dir='workflow_outputs_finqabench_aapl10k'):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    results = []
    
    for index, row in df.iterrows():
        if index < 14:
            continue
        query_input = row['Query']  # Assuming the column is named 'question'
        print(f"Processing query {index}: {query_input}")
        print("=========================================")
        
        global tokens, inp_tok
        tokens = 0
        inp_tok=0
        
        # Decompose the input query into sub-queries
        inputs = {"question": query_input, "revision_number": 0, "max_revisions": 1}
        
        with get_openai_callback() as cb:
            subQueries = decomposer.invoke(inputs).sub_queries
        
        tokens += cb.total_tokens
        inp_tok += cb.prompt_tokens
        
        # If no sub-queries, use original query
        if not subQueries:
            subQueries = [query_input]
        
        # Generate outputs for the sub-queries
        generations = await generator(subQueries, verification=" ")
        
        # Aggregate answers
        answers = [gen["generation"] for gen in generations]
        
        with get_openai_callback() as cb:
            tokens += cb.total_tokens
            inp_tok += cb.prompt_tokens
            final_answer = aggregator.invoke({
                "question": query_input, 
                "answers": ", ".join(answers)
            }).answer
        
        # Store results
        result_entry = {
            "original_query": query_input,
            "sub_queries": subQueries,
            "sub_query_results": [gen["generation"] for gen in generations],
            "final_answer": final_answer,
            "total_tokens": tokens,
            "input_tokens": inp_tok,
            "output_tokens": tokens - inp_tok
        }
        
        # Generate a safe filename
        safe_filename = f"{index}_" + "".join(x for x in query_input if x.isalnum() or x in [' ', '_']).rstrip()
        safe_filename = safe_filename[:20]  # Limit filename length
        json_path = os.path.join(output_dir, f"{index}_{safe_filename}.json")
        
        # Save individual JSON file
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(result_entry, f, indent=2, ensure_ascii=False)
        
        results.append(result_entry)
    
    return results


# Process queries (you'll need to pass your actual decomposer, generator, and aggregator)
results = await process_dataframe_queries(question_df, decomposer, Generator, aggregator)


### Naive-RAG with Contextual

In [4]:
import json
import time
import os
import pandas as pd
from dotenv import load_dotenv
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.vectorstores import PathwayVectorClient
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import tiktoken  # For token counting

# Load environment variables
def load_environment_variables():
    """Load environment variables from a .env file."""
    print("Loading environment variables...")
    load_dotenv()
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    print("Environment variables loaded.")

# Initialize Pathway retriever
def create_pathway_client(host="127.0.0.1", port=8011, k=10):
    """Initialize PathwayVectorClient and return as retriever."""
    print(f"Initializing PathwayVectorClient on host {host} and port {port}...")
    client = PathwayVectorClient(host=host, port=port)
    print("PathwayVectorClient initialized.")
    return client.as_retriever(search_kwargs={"k": k})

# Retrieve relevant documents
def retrieve_relevant_documents(retriever, query):
    """Retrieve relevant documents based on the given query."""
    print(f"Retrieving relevant documents for query: {query}")
    try:
        docs = retriever.get_relevant_documents(query)
        print(f"Retrieved {len(docs)} chunks.")
        return docs
    except Exception as e:
        print(f"Error retrieving documents: {e}")
        return []

# Create document compressor
def create_compressor(model="rerank-english-v3.0"):
    """Create and return a CohereRerank compressor."""
    print("Creating CohereRerank compressor...")
    compressor = CohereRerank(model=model, top_n=5)
    print("Compressor created.")
    return compressor

# Compress retrieved documents
def compress_documents(retriever, query, compressor):
    """Compress relevant documents using the provided compressor."""
    print("Compressing retrieved chunks...")
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever
    )
    try:
        compressed_docs = compression_retriever.invoke(query)
        print(f"Compressed to {len(compressed_docs)} chunks.")
        return compressed_docs
    except Exception as e:
        print(f"Error during compression: {e}")
        return []

# Initialize OpenAI LLM
def create_llm():
    """Create and return an OpenAI LLM instance."""
    print("Initializing OpenAI LLM...")
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        max_tokens=None,
        timeout=45,
        max_retries=2,
    )
    print("OpenAI LLM initialized.")
    return llm

# Create ChatPromptTemplate for Q&A
def create_chat_prompt():
    """Create and return a chat prompt template for Q&A."""
    print("Creating chat prompt template...")
    chat_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful assistant. Answer the following question based only on the provided context.\n"
                "Provide a step-by-step explanation, and reference the source details using metadata.\n\n"
                "<context>\n{context}\n</context>",
            ),
            ("human", "{question}"),
        ]
    )
    print("Chat prompt template created.")
    return chat_prompt

# Token counting function
def count_tokens(text, model="o200k_base"):
    """Estimate token count for given text using tiktoken."""
    tokenizer = tiktoken.get_encoding(model)
    tokens = tokenizer.encode(text)
    return len(tokens)

# Generate answer
def get_answer(context, question, chain):
    """Generate an answer based on the context and question using the LLM chain."""
    print("Generating answer for the question...")
    try:
        input_tokens = count_tokens(context) + count_tokens(question)
        print(f"Estimated input tokens: {input_tokens}")

        # Invoke the LLM chain with structured input
        answer = chain.invoke({"context": context, "question": question})
        
        output_tokens = count_tokens(answer.content)
        print(f"Estimated output tokens: {output_tokens}")
        
        return answer.content
    except Exception as e:
        print(f"Error generating answer: {e}")
        return None

# Main execution logic
def main():
    # Load environment variables
    load_environment_variables()

    # Load the dataset
    file_path = 'rag_benchmark_apple_10k_2022_with_context.csv'
    print(f"Loading dataset from {file_path}...")
    data = pd.read_csv(file_path)
    print(f"Loaded {len(data)} questions.")

    # Initialize Pathway retriever, compressor, LLM, and chat prompt
    retriever = create_pathway_client()
    compressor = create_compressor()
    llm = create_llm()
    chat_prompt = create_chat_prompt()

    # Create the LLM chain
    chain = chat_prompt | llm

    # Prepare results storage
    results = []
    print("Total number of rows: ", len(data))
    # Loop over each question in the dataset
    for idx, row in data.iterrows():
        question = row['Query']
        gt_answer = row.get('Response', "Ground truth answer not provided")
        
        print(f"\nProcessing question ID {idx}: {question}")

        # Step 1: Retrieve relevant documents
        retrieved_docs = retrieve_relevant_documents(retriever, question)
        if not retrieved_docs:
            print(f"No documents retrieved for query ID {idx}.")
            continue

        # Step 2: Compress retrieved documents
        compressed_docs = compress_documents(retriever, question, compressor)
        # compressed_docs = retrieved_docs
        if compressed_docs == []:
            compressed_docs = retrieved_docs

        # Prepare the context from compressed documents
        context = "\n".join([doc.page_content for doc in compressed_docs if doc.page_content])

        # Step 3: Generate an answer with retry logic
        answer = get_answer(context, question, chain)
        if answer is None:
            print(f"Retrying for question ID {idx} due to no initial response...")
            time.sleep(50)  # Wait 50 seconds before retrying
            answer = get_answer(context, question, chain)

        # Structure retrieved context in a list of dictionaries as per your format
        retrieved_context = [
            {"doc_id": doc.metadata.get("doc_id", "unknown"), "text": doc.page_content}
            for doc in compressed_docs
        ]
        
        # Append to results
        results.append({
            "query_id": str(idx).zfill(3),
            "question": question,
            "gt_answer": gt_answer,
            "response": answer,
            "retrieved_context": retrieved_context
        })

    # Save the results to a JSON file
    output_path = 'naive_rag_contextual_parser_apple_10k_2022.json'
    print(f"Saving evaluation results to {output_path}...")
    with open(output_path, 'w') as f:
        json.dump({"results": results}, f, indent=4)

    print(f"Evaluation results saved to {output_path}")

# Run main function
if __name__ == "__main__":
    main()

### Naive RAG without Contextual

In [5]:
import json
import time
import os
import pandas as pd
from dotenv import load_dotenv
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.vectorstores import PathwayVectorClient
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import tiktoken  # For token counting

# Load environment variables
def load_environment_variables():
    """Load environment variables from a .env file."""
    print("Loading environment variables...")
    load_dotenv()
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    print("Environment variables loaded.")

# Initialize Pathway retriever
def create_pathway_client(host="127.0.0.1", port=8011, k=10):
    """Initialize PathwayVectorClient and return as retriever."""
    print(f"Initializing PathwayVectorClient on host {host} and port {port}...")
    client = PathwayVectorClient(host=host, port=port)
    print("PathwayVectorClient initialized.")
    return client.as_retriever(search_kwargs={"k": k})

# Retrieve relevant documents
def retrieve_relevant_documents(retriever, query):
    """Retrieve relevant documents based on the given query."""
    print(f"Retrieving relevant documents for query: {query}")
    try:
        docs = retriever.get_relevant_documents(query)
        print(f"Retrieved {len(docs)} chunks.")
        return docs
    except Exception as e:
        print(f"Error retrieving documents: {e}")
        return []

# Create document compressor
def create_compressor(model="rerank-english-v3.0"):
    """Create and return a CohereRerank compressor."""
    print("Creating CohereRerank compressor...")
    compressor = CohereRerank(model=model, top_n=5)
    print("Compressor created.")
    return compressor

# Compress retrieved documents
def compress_documents(retriever, query, compressor):
    """Compress relevant documents using the provided compressor."""
    print("Compressing retrieved chunks...")
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever
    )
    try:
        compressed_docs = compression_retriever.invoke(query)
        print(f"Compressed to {len(compressed_docs)} chunks.")
        return compressed_docs
    except Exception as e:
        print(f"Error during compression: {e}")
        return []

# Initialize OpenAI LLM
def create_llm():
    """Create and return an OpenAI LLM instance."""
    print("Initializing OpenAI LLM...")
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        max_tokens=None,
        timeout=45,
        max_retries=2,
    )
    print("OpenAI LLM initialized.")
    return llm

# Create ChatPromptTemplate for Q&A
def create_chat_prompt():
    """Create and return a chat prompt template for Q&A."""
    print("Creating chat prompt template...")
    chat_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful assistant. Answer the following question based only on the provided context.\n"
                "Provide a step-by-step explanation, and reference the source details using metadata.\n\n"
                "<context>\n{context}\n</context>",
            ),
            ("human", "{question}"),
        ]
    )
    print("Chat prompt template created.")
    return chat_prompt

# Token counting function
def count_tokens(text, model="o200k_base"):
    """Estimate token count for given text using tiktoken."""
    tokenizer = tiktoken.get_encoding(model)
    tokens = tokenizer.encode(text)
    return len(tokens)

# Generate answer
def get_answer(context, question, chain):
    """Generate an answer based on the context and question using the LLM chain."""
    print("Generating answer for the question...")
    try:
        input_tokens = count_tokens(context) + count_tokens(question)
        print(f"Estimated input tokens: {input_tokens}")

        # Invoke the LLM chain with structured input
        answer = chain.invoke({"context": context, "question": question})
        
        output_tokens = count_tokens(answer.content)
        print(f"Estimated output tokens: {output_tokens}")
        
        return answer.content
    except Exception as e:
        print(f"Error generating answer: {e}")
        return None

# Main execution logic
def main():
    # Load environment variables
    load_environment_variables()

    # Load the dataset
    file_path = 'rag_benchmark_apple_10k_2022_with_context.csv'
    print(f"Loading dataset from {file_path}...")
    data = pd.read_csv(file_path)
    print(f"Loaded {len(data)} questions.")

    # Initialize Pathway retriever, compressor, LLM, and chat prompt
    retriever = create_pathway_client()
    compressor = create_compressor()
    llm = create_llm()
    chat_prompt = create_chat_prompt()

    # Create the LLM chain
    chain = chat_prompt | llm

    # Prepare results storage
    results = []
    print("Total number of rows: ", len(data))
    # Loop over each question in the dataset
    for idx, row in data.iterrows():
        question = row['Query']
        gt_answer = row.get('Response', "Ground truth answer not provided")
        
        print(f"\nProcessing question ID {idx}: {question}")

        # Step 1: Retrieve relevant documents
        retrieved_docs = retrieve_relevant_documents(retriever, question)
        if not retrieved_docs:
            print(f"No documents retrieved for query ID {idx}.")
            continue

        # Step 2: Compress retrieved documents
        compressed_docs = compress_documents(retriever, question, compressor)
        if compressed_docs == []:
            compressed_docs = retrieved_docs

        # Prepare the context from compressed documents
        context = "\n".join([doc.page_content for doc in compressed_docs if doc.page_content])

        # Step 3: Generate an answer with retry logic
        answer = get_answer(context, question, chain)
        if answer is None:
            print(f"Retrying for question ID {idx} due to no initial response...")
            time.sleep(50)  # Wait 50 seconds before retrying
            answer = get_answer(context, question, chain)

        # Structure retrieved context in a list of dictionaries as per your format
        retrieved_context = [
            {"doc_id": doc.metadata.get("doc_id", "unknown"), "text": doc.page_content}
            for doc in compressed_docs
        ]
        
        # Append to results
        results.append({
            "query_id": str(idx).zfill(3),
            "question": question,
            "gt_answer": gt_answer,
            "response": answer,
            "retrieved_context": retrieved_context
        })

    # Save the results to a JSON file
    output_path = 'naive_rag_without_contextual_parser_apple_10k_2022.json'
    print(f"Saving evaluation results to {output_path}...")
    with open(output_path, 'w') as f:
        json.dump({"results": results}, f, indent=4)

    print(f"Evaluation results saved to {output_path}")

# Run main function
if __name__ == "__main__":
    main()